![EvalML Logo](images/evalml_logo.png)

# What is EvalML
EvalML is an AutoML library to build optimized machine learning pipelines for domain-specific objective functions.


Combined with `Featuretools <www.featuretools.com>`__ and `Compose <compose.ml>`__, EvalML can be used to create a state-of-the-art end-to-to end machine learning solutions.


# Quick Start Example

In [1]:
%load_ext autoreload
%autoreload 2
import evalml

## Load Data

First, we load in the features and targets we want to use to train our model

In [2]:
X, y = evalml.demos.load_breast_cancer()

## Configure search

EvalML has many options to configure the pipeline search. At the minimum, we need to define an objective function. For simplicity, we will use the F1 score in this example as the objective function. However, the real power of EvalML is in using [domain-specific]() objective functions or [building your own](). 

In [3]:
clf = evalml.AutoClassifier(objective="f1",
                            max_pipelines=5)

In order to validate the results of the pipeline creation and optimization process, we will save some of our data as a holdout set

In [4]:
X_train, X_holdout, y_train, y_holdout = evalml.preprocessing.split_data(X, y, test_size=.2)

When we call `.fit()`, the search for the best pipeline will begin. 

In [ ]:
clf.fit(X_train, y_train)

*****************************
* Beginning pipeline search *
*****************************

Optimizing for F1. Greater score is better.

Searching up to 5 pipelines. No time limit is set. Set one using max_time parameter.

Possible model types: random_forest, xgboost, linear_model

Testing Random Forest w/ imputation:   0%|          | 0/5 [00:00<?, ?it/s]

## See Pipeline Rankings

After the search is finished we can view all of the pipelines searched, ranked by score. Internally, EvalML performs cross validation to score the pipelines. If it notices, a high variance across cross validation folds, it will warn you.

In [ ]:
clf.rankings

## Describe pipeline

If we are interested in see more details about the pipeline we can describe it using the `id` from the rankings table

In [ ]:
clf.describe_pipeline(3)

## Select Best pipeline
We can now select best pipeline and score it on our holdout data

In [ ]:
pipeline = clf.best_pipeline
pipeline.score(X_holdout, y_holdout)

# Getting Started
[What is EvalML](self)

[Install](install)

# Overview
[Pipeline Search](overview/pipeline_search)

[Objective  Functions](overview/objective_functions)

# Demos

[Fraud Prediction](demos/fraud)

[Lead Scoring](demos/lead_scoring)

# Resources
[API Reference](api_reference)